step 1 load the workspace configuration

In [28]:
from azureml.core import Workspace , Environment, Dataset,Experiment
subscription_id = 'd5d1b5f7-5210-4d65-8f0c-730767acb30b'
resource_group = 'imdrkapil-rg'
workspace_name = 'mlworkspaceaug'

workspace = Workspace(subscription_id, resource_group, workspace_name)


step 2 start with experiment

In [13]:
experiment=Experiment(workspace=workspace, name='iris')
dataset = Dataset.get_by_name(workspace, name='iris')
df=dataset.to_pandas_dataframe()
df.head(4)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa


In [31]:
import joblib
from sklearn.metrics.cluster import entropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
x = df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
y = df['Species']
run=experiment.start_logging()
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,shuffle=True)
classifier = DecisionTreeClassifier(criterion="entropy")
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy*100)
run.log('accuracy',accuracy*100)
model_name='iris_decision_tree.pk1'
filepath='outputs/'+model_name
joblib.dump(value=classifier, filename=filepath)
run.upload_file(name=model_name, path_or_stream=filepath)
run.complete()

95.55555555555556


# registering the model

In [33]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
import sklearn


In [34]:
model = Model.register(
    workspace=workspace,
    model_name = "iris",
    model_path = filepath,
    model_framework= Model.Framework.SCIKITLEARN,
    model_framework_version= sklearn.__version__,
    description= "This is a sample DT classifier",
    resource_configuration= ResourceConfiguration(cpu=1, memory_in_gb= 0.5)
)

print("Name:", model.name)
print("version:", model.version)

Registering model iris
Name: iris
version: 1
